### Packages

In [1]:
using DynamicalSystems, DifferentialEquations, OrdinaryDiffEq, StaticArrays
using CairoMakie

### Functions

In [50]:
function model(u, p ,t)
    
    E, x, u_, y = u
    τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0 = p
    
    U(y, U0, ΔU0, ythr) = U0 + ΔU0 / ( 1 + exp( -50 * ( y - ythr ) ) )
    σ(x, xthr)= 1 / (1 + exp(-20 * (x - xthr)));
    
    du1 = (-E + α * log( 1 + exp( ( J * u_ * x * E + I0 ) / α ) )) / τ
    du2 = ( 1 - x ) / τ_D - u_ * x * E
    du3 = (U(y, U0, ΔU0, ythr) - u_) / τ_F + U(y, U0, ΔU0, ythr) * ( 1 - u_ ) * E
    du4 = -y /  τ_y + β * σ(x, xthr)
    
    return SVector(du1, du2, du3, du4)
    
end;

function output(index_i, index_j, U0_lc, I0_lc, u0_lc)
        println("index i: $index_i"); flush(stdout)
        println("index j: $index_j"); flush(stdout)
        println("U0: $U0_lc;"); flush(stdout)
        println("I0: $I0_lc;"); flush(stdout)
        println("u0: $u0_lc;"); flush(stdout)
end
function separate()
    println(" --------------------- "); flush(stdout)
    println("  "); flush(stdout)
end

function Y(x)
    a = 0.1836121365412579
    b = 0.2749462902187088
    c = 0.25
    return  a * (x.^2) + b * x .+ c 
end

function print_output(index, I0, U0, initial_condition)
    println("index: $index");flush(stdout)
    println("I0: $I0");flush(stdout)
    println("U0: $U0");flush(stdout)
    println("initial condition: $initial_condition")
end
function separate()
    println("_________________")
    println("")
end

separate (generic function with 1 method)

### Parametres

In [45]:
τ = 0.013; τ_D = 0.15; τ_y = 1.8; τ_F = 1;
α = 1.5; J = 3.07; β = 0.4375
ΔU0 = 0.305
xthr = 0.9; ythr = 0.5
parametres = SA[ τ, α, τ_D, J, NaN, ΔU0, τ_y, β, xthr, ythr, NaN ]
timefinal = 500.0
println(parametres)

[0.013, 1.5, 0.15, 3.07, NaN, 0.305, 1.8, 0.4375, 0.9, 0.5, NaN]


### Grid

In [91]:
startI = -1.8
endI = -1.24798
lengthI = 100

rangeI = range( startI, endI, length = lengthI )
rangeU0 = Y( rangeI );

In [92]:
file = "output_from_function.txt"

"output_from_function.txt"

In [ ]:
for (index, I0) in enumerate(rangeI)
    """
        Итерируемся по точкам c функции
    """
    if index == 1
        global initial_condition_function = SA[0.0, 0.2, 0.1, 0.1]
    end
    
    U0 = Y(I0)
    
    print_output(index, I0, U0, initial_condition_function)
    
    parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0  ]
    ds = ContinuousDynamicalSystem( model, initial_condition_function, parametres )
    
    tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
    
    initial_condition_function = tr[end, :]
    println("last point: $initial_condition_function");flush(stdout)
    λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
    println("LSE: $λλ");flush(stdout)

    open(file, "a") do io
        println(io, " $I0 $U0 $(λλ[1])")
    end
    
    firstdot( I0, U0, initial_condition_function )
    lastdot( I0, U0, initial_condition_function )
    k_dot( index, I0, U0, initial_condition_function )
    separate()
end

index: 1
I0: -1.8
U0: 0.34999999999999987


In [107]:
function firstdot( I0, U0, initial_condition_function )
    
    if I0 == startI
        println("");flush(stdout)
        println("FIRST ITERATION");flush(stdout)
        println("MOVE TO RIGHT")
        step_to_right = rangeI.step.hi
        range_to_right_border = range( I0 + step_to_right, endI, step = step_to_right )
    
        for (index, I0_right) in enumerate(range_to_right_border)
        
            if index == 1
                global initial_condition_right = initial_condition_function
            end
            println("->")
            print_output(index, I0_right, U0, initial_condition_right)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_right  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_right, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_right = tr[end, :]
        
            println("last point: $initial_condition_right");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)

            open(file, "a") do io
                println(io, " $I0_right $U0 $(λλ[1])")
            end
            separate()
        end
    end
end

firstdot (generic function with 1 method)

In [108]:
function lastdot( I0, U0, initial_condition_function )
    
    if I0 == endI
        println("");flush(stdout)
        println("LAST ITERATION");flush(stdout)
        println("MOVE TO LEFT")
        step_to_left = rangeI.step.hi
        range_to_left_border = range( endI - step_to_left, startI, step = -step_to_left )
    
        for (index, I0_left) in enumerate(range_to_left_border)
        
            if index == 1
                global initial_condition_left = initial_condition_function
            end
            println("<-")
            print_output(index, I0_left, U0, initial_condition_left)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_left = tr[end, :]
        
            println("last point: $initial_condition_left");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)

            open(file, "a") do io
                println(io, " $I0_left $U0 $(λλ[1])")
            end
            separate()
        end
    end
end

lastdot (generic function with 1 method)

In [109]:
function k_dot( index_I0, I0, U0, initial_condition_function )
    println("");flush(stdout)
    println(" K-dot: $index_I0 ");flush(stdout)
    # move to left
    step_to_left = rangeI.step.hi
    range_to_left_border = range( I0 - step_to_left, startI, step = -step_to_left )
    
    for (index, I0_left) in enumerate(range_to_left_border)
        
            if index == 1
                global initial_condition_left = initial_condition_function
            end
            println("<-")
            print_output(index, I0_left, U0, initial_condition_left)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_left = tr[end, :]
        
            println("last point: $initial_condition_left");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)

            open(file, "a") do io
                println(io, " $I0_left $U0 $(λλ[1])")
            end
            separate()
    end
    
    #move to right
    step_to_right = rangeI.step.hi
    range_to_right_border = range( I0 + step_to_right, endI, step = step_to_right )
    for (index, I0_left) in enumerate(range_to_left_border)
        
            if index == 1
                global initial_condition_left = initial_condition_function
            end
            println("<-")
            print_output(index, I0_left, U0, initial_condition_left)
        
            parametres = SA[  τ, α, τ_D, J, U0, ΔU0, τ_y, β, xthr, ythr, I0_left  ]
        
            ds = ContinuousDynamicalSystem( model, initial_condition_left, parametres )
    
            tr = trajectory(ds, timefinal, Ttr = timefinal, Δt = 0.001;
                    diffeq = (alg = RK4(), adaptive = false, dense = false, dt = 0.001))
            initial_condition_left = tr[end, :]
        
            println("last point: $initial_condition_left");flush(stdout)
            λλ = lyapunovspectrum( ds, timefinal, Ttr = timefinal, Δt = 1;
                            diffeq = ( alg = RK4(), adaptive = false, dense = false, dt = 0.001 ))
            println("LSE: $λλ");flush(stdout)

            open(file, "a") do io
                println(io, " $I0_left $U0 $(λλ[1])")
            end
            separate()
        end
end

k_dot (generic function with 2 methods)